<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_041_langchain_router_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  LLMRouterChain

In [2]:
# !pip install langchain
# !pip install openai
# !pip install python-dotenv
# !pip install langchain-openai

In [3]:
import os
from dotenv import load_dotenv
import openai
import json
import langchain
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# Load environment variables from .env file
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
# Set the environment variable globally for libraries like LangChain
os.environ["OPENAI_API_KEY"] = api_key
# Print the API key to confirm it's loaded correctly
print("API Key loaded from .env:",os.environ["OPENAI_API_KEY"][0:30])

API Key loaded from .env: sk-proj-e1GUWruINPRnrozmiakkRM


### Route Templates

In [8]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms.
You assume no prior knowledge. Here is the question\n{input}'''

expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a
PhD level understanding of Physics. Here is the question\n{input}'''

### Route Prompts

In [9]:
prompt_infos = [
    {'name':'advanced physics','description': 'Answers advanced physics questions',
     'prompt_template':expert_template},
    {'name':'beginner physics','description': 'Answers basic beginner physics questions',
     'prompt_template':beginner_template},

]

### ConversationChain

In [17]:
# Define the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [45]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

print(destination_chains.keys())

dict_keys(['advanced physics', 'beginner physics'])


### Destination Chains

In [20]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm,prompt=default_prompt)

### Multi Routing Template

In [21]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [22]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

### Routing Destinations


In [23]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [24]:
print(destinations_str)

advanced physics: Answers advanced physics questions
beginner physics: Answers basic beginner physics questions


### Router Prompt

In [25]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [26]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [27]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
advanced physics: Answers advanced physics questions
beginner physics: Answers basic beginner physics

### Routing Chain Call

In [31]:
from langchain.chains.router import MultiPromptChain
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [32]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [33]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [35]:
chain.invoke("How do magnets work?")



> Entering new MultiPromptChain chain...
beginner physics: {'input': 'How do magnets work?'}
> Finished chain.


{'input': 'How do magnets work?',
 'text': "Great question! Let's break it down in simple terms.\n\n### What Are Magnets?\n\nMagnets are objects that can attract (pull) or repel (push away) certain materials, like iron. You might have seen them in action when they stick to your fridge or when they hold papers together.\n\n### What Makes a Magnet?\n\n1. **Atoms and Electrons**: Everything around us is made up of tiny particles called atoms. Atoms have smaller particles inside them, called electrons. These electrons move around the atom, and this movement creates tiny magnetic fields.\n\n2. **Magnetic Domains**: In most materials, the magnetic fields of the atoms point in all different directions, and they cancel each other out. But in magnets, many of these tiny magnetic fields line up in the same direction. When enough of them align, the material becomes a magnet.\n\n### How Do Magnets Work?\n\n1. **Poles**: Every magnet has two ends called poles: a north pole and a south pole. The nor

### Full Code

### **Explanation of What the Code is Doing**

1. **Templates for Task-Specific Responses**:
   - Two templates (`beginner_template` and `expert_template`) are defined for answering physics questions tailored to different audiences:
     - Beginner: Simplified, no prior knowledge required.
     - Advanced: Detailed and assumes expertise.

2. **Destination Chains**:
   - Each prompt template is turned into a task-specific chain using `LLMChain`.
   - These chains are stored in a dictionary (`destination_chains`) and keyed by their respective task names (`beginner physics`, `advanced physics`).

3. **Default Chain**:
   - A fallback chain handles inputs that don’t match any specific task. This uses a generic prompt.

4. **Router Template**:
   - The `MULTI_PROMPT_ROUTER_TEMPLATE` provides the structure for how the LLM decides the appropriate chain for an input.
   - It lists available task descriptions and dynamically evaluates which task the input belongs to.

5. **Router Chain**:
   - Uses the router prompt and LLM to classify inputs and map them to a specific destination chain (e.g., "advanced physics").
   - Outputs the task name corresponding to the most relevant chain.

6. **MultiPromptChain**:
   - Combines the router chain, destination chains, and default chain.
   - It routes inputs dynamically to the correct chain and executes the associated task.

7. **Invocation**:
   - The `invoke` method is called with a question: `"How do magnets work?"`
   - The router determines if the question is for a beginner or advanced audience and forwards it to the appropriate chain.

---

### **Key Concepts and Lessons**
1. **Dynamic Task Routing**:
   - The router chain ensures inputs are processed by the correct chain dynamically.
2. **Separation of Logic**:
   - Task-specific chains are modular and reusable.
3. **Scalability**:
   - New tasks or chains can be easily added by extending the `prompt_infos` list.
4. **Fallback Mechanism**:
   - The default chain ensures robustness when no specific task matches the input.


In [48]:
from langchain.chains.router import MultiPromptChain
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# Define a template for beginners' questions
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms.
You assume no prior knowledge. Here is the question\n{input}'''

# Define a template for advanced questions
expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a
PhD level understanding of Physics. Here is the question\n{input}'''

# List of prompt configurations for different levels of physics questions
prompt_infos = [
    {
        'name': 'advanced physics',  # Identifier for advanced questions
        'description': 'Answers advanced physics questions',  # Task description
        'prompt_template': expert_template  # Associated prompt template
    },
    {
        'name': 'beginner physics',  # Identifier for beginner questions
        'description': 'Answers basic beginner physics questions',  # Task description
        'prompt_template': beginner_template  # Associated prompt template
    },
]

# Step 1: Initialize the Language Model (LLM)
# The ChatOpenAI instance specifies the model and behavior (temperature controls creativity)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Step 2: Create chains for each prompt configuration
# Initialize an empty dictionary to store chains for each task
destination_chains = {}

# Loop through the prompt configurations and create a chain for each
for p_info in prompt_infos:
    name = p_info["name"]  # Retrieve the task name (e.g., "advanced physics")
    prompt_template = p_info["prompt_template"]  # Retrieve the corresponding prompt template
    prompt = ChatPromptTemplate.from_template(template=prompt_template)  # Create a ChatPromptTemplate
    chain = LLMChain(llm=llm, prompt=prompt)  # Create an LLMChain using the prompt and LLM
    destination_chains[name] = chain  # Store the chain in the dictionary with the task name as the key

# Step 3: Define a default prompt and chain
# This is used if no specific task is matched during routing
default_prompt = ChatPromptTemplate.from_template("{input}")  # Basic template for fallback
default_chain = LLMChain(llm=llm, prompt=default_prompt)  # Create the fallback chain

# Step 4: Print the Multi-Prompt Router Template
# This template is used to dynamically decide which chain to use based on input
print(MULTI_PROMPT_ROUTER_TEMPLATE)

# Generate a string listing all available task destinations with descriptions
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)  # Combine all destinations into a formatted string
print(destinations_str)  # Print the formatted string for debugging or review

# Import additional utilities for the router chain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

# Step 5: Create a router prompt
# This prompt is used by the LLM to decide which chain should handle the input
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str  # Insert the destination descriptions into the router template
)
router_prompt = PromptTemplate(
    template=router_template,  # Use the formatted router template
    input_variables=["input"],  # Input to be classified
    output_parser=RouterOutputParser(),  # Parse the LLM's classification result
)

# Print the router template for debugging
print(router_template)

# Step 6: Create the router chain
# This chain uses the router prompt and the LLM to classify inputs and decide which chain to invoke
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# Step 7: Create the MultiPromptChain
# Combines the router chain, destination chains, and default chain into a single workflow
chain = MultiPromptChain(
    router_chain=router_chain,  # Router chain for task classification
    destination_chains=destination_chains,  # Task-specific chains
    default_chain=default_chain,  # Fallback chain
    verbose=True  # Enable detailed logging of the workflow
)

# Step 8: Invoke the MultiPromptChain with an input
# Dynamically routes the question to the appropriate chain based on its content
chain.invoke("How do magnets work?")


Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

{'input': 'How do magnets work?',
 'text': 'Great question! Let’s break it down step by step.\n\n**1. What are Magnets?**\nMagnets are objects that can attract or repel certain materials, mainly iron, nickel, and cobalt. They have two ends called poles: a north pole and a south pole.\n\n**2. The Basics of Magnetism:**\nAt the core of every magnet is something called "magnetic fields." You can think of a magnetic field as an invisible force field that surrounds the magnet and extends out into the space around it. This field is what causes magnets to attract or repel other magnets or magnetic materials.\n\n**3. How Do Magnets Attract or Repel?**\n- **Attraction:** When you bring the north pole of one magnet close to the south pole of another magnet, they pull towards each other. This is because opposite poles attract.\n- **Repulsion:** If you bring two north poles or two south poles close together, they push away from each other. This is because like poles repel.\n\n**4. Why Do Magnets H

In [ ]:
chain.invoke("How do Feynman Diagrams work?")



> Entering new  chain...
advanced physics: {'input': 'How do Feynman Diagrams work?'}
> Finished chain.


'Feynman diagrams are graphical representations used in particle physics to visualize and calculate the interactions between elementary particles. They were developed by the physicist Richard Feynman as a powerful tool for understanding and calculating the behavior of subatomic particles based on quantum field theory.\n\nAt their core, Feynman diagrams depict the possible ways in which particles can interact and exchange energy and momentum. They provide a pictorial representation of the mathematical expressions that describe these interactions.\n\nIn a Feynman diagram, particles are represented by lines, with arrows indicating their direction of motion. The lines can be straight or wavy, representing different types of particles. For example, straight lines can represent fermions (such as electrons or quarks), while wavy lines can represent force-carrying bosons (such as photons or gluons).\n\nThe vertices where the lines meet in a Feynman diagram represent interactions between partic